The standard neural network we looked at in the previous lesson takes in a vector as input thus a flattened image could be passed in as input and used for classification problems successfully. But this is not the best way to do it. If you think about an image, the spatial relations between the different pixels is an important piece of information when determening what the image is. If we scrambled the pixels of an image, it would be much harder to determine what is in it. This is basically what we are doing when we flatten the image and use standard NNs as the order of pixels which we use in the input vector to the network does not affect the performance of the NN. We are losing the information of the spatial relations of the pixels.

Convolutional neural networks solve this very problem. Rather than performing a matrix multiplication, a convolution operation is performed which can take in a 2d input and give a 2d output hence keep the information about the spatial relations of the pixels. This greatly increases their performance on image and video processing tasks.

In the convolution proccess, you have a filter which you start on the top left side of the image and slide across the whole image, taking a dot product between the values of the filter and pixel values of the image. Bear in mind that colour images have three channels so your filter may be 3-d so you take the dot product across a 3d-volume. Each dot product corresponds to a single activation value in a 2-d matrix of neurons which corresponds to a single layer in the output.

The animation below shows how a 1x3x3 filter is applied to a 1x5x5 image. Notice how the output has high output values when the filter is passed over locations where there is an X shape in the input image. This is because the values of the filter are such that it is performing pattern matching for the X shape.


For a long time, operations like this were used in computer vision to find different patterns in images with the engineers having to manually tune the values of the filters to perform the required function. The only difference now is that we apply an activation such as Relu or Sigmoid at each layer and after setting up the structure of the network, we initialize the filter values randomly before using gradient descent to automatically tune the values of the filters. We can also apply pooling operations to subsample the output at each layer therefore reducing the number of parameters that need to be learned for the succeeding convolution operation.


Just like before, each layer in the whole network learns higher level abstract features from the inputs. In CNNs the features are even more interpretable as the output of each layer is 2d so can be viewed as an image.

![image](images/convolution_animation.gif)
![image](images/convolution_image.jpg)

subset of fcnn

We have some prior understanding of this data. We know it is an image so we want our output to be translation invariant. riors on the search 

two particularly useful proprties about an image 
if you flatten image, and it turns into size 100 vector, 

translated image leads to significantly worse results for fcnn compared to cnn. we want to look for the same features at different regions of the image




In [ ]:
#test_input = 

In [10]:
import torch
from torchvision import datasets, transforms

# GET THE TRAINING DATASET
train_data = datasets.MNIST(root='MNIST-data',                        # where is the data (going to be) stored
                            transform=transforms.ToTensor(),          # transform the data from a PIL image to a tensor
                            train=True,                               # is this training data?
                            download=True                             # should i download it if it's not already here?
                           )

# GET THE TEST DATASET
test_data = datasets.MNIST(root='MNIST-data',
                           transform=transforms.ToTensor(),
                           train=False,
                          )

# PRINT THEIR LENGTHS AND VISUALISE AN EXAMPLE
x = train_data[0][0]    # get the first example
#tensor = # get the actual input data
#t = # create the transform that can be called to convert the tensor into a PIL Image
#img = t(img)    # call the transform on the tensor
#img.show()    # show the image

In [11]:
x.shape

torch.Size([1, 28, 28])

In [ ]:
.# FURTHER SPLIT THE TRAINING INTO TRAINING AND VALIDATION
train_data, val_data = torch.utils.data.random_split(train_data, [50000, 10000])    # split into 50K training & 10K validation

In [ ]:
batch_size = 256

# MAKE TRAINING DATALOADER
train_loader = torch.utils.data.DataLoader(
    train_data,
    shuffle=True,
    batch_size=batch_size
)

# MAKE VALIDATION DATALOADER
val_loader = torch.utils.data.DataLoader(
    val_data,
    shuffle=True,
    batch_size=batch_size
)

# MAKE TEST DATALOADER
test_loader = torch.utils.data.DataLoader(
    test_data,
    shuffle=True,
    batch_size=batch_size
)

In [ ]:
class convnet(torch.nn.Module):
    def __init__(self):
        super().__init__()
            # conv2d(in_channels, out_channels, kernel_size)
            # in_channels is the number of layers which it takes in (i.e.num color streams in 1st layer)
            # out_channels is the number of different filters that we use
            # kernel_size is the depthxwidthxheight of the kernel#
            # stride is how many pixels we shift the kernel by each time
        self.conv_layers = torch.nn.Sequential( # put your convolutional architecture here using torch.nn.Sequential 
            torch.nn.Conv2d(1, 16, kernel_size=5, stride=1),
            torch.nn.Conv2d(16, 32, kernel_size=5, stride=1)
        )
        self.fc_layers = torch.nn.Sequential(
            self.fc1 = torch.nn.Linear(12800, 10) # put your convolutional architecture here using torch.nn.Sequential 
        )
    def forward(self, x):
        x = self.conv_layers(x)# pass through conv layers
        x = x.view(x.shape[0], -1)# flatten output ready for fully connected layer
        x = self.fc_layers(x)# pass through fully connected layer
        x = F.softmax(x, dim=1)# softmax activation function on outputs
        return x

In [ ]:
learning_rate = 0.001 # set learning rate
epochs = 3# set number of epochs

cnn = convnet()#instantiate model
criterion = torch.nn.CrossEntropyLoss() #use cross entropy loss function
optimizer = torch.optim.Adam() # use Adam optimizer, passing it the parameters of your model and the learning rate

# SET UP TRAINING VISUALISATION
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter() # we will use this to show our models performance on a graph

In [ ]:
def train(epochs):    
    for epoch in range(epochs):
        for idx, (inputs, labels) in enumerate(train_loader):
            # pass x through your model to get a prediction
            prediction = model(inputs)             # pass the data forward through the model
            loss = criterion(prediction, labels)   # compute the cost
            print('Epoch:', epoch, '\tBatch:', idx, '\tLoss:', loss.data[0])
            
            optimiser.zero_grad()                  # reset the gradients attribute of all of the model's params to zero
            loss.backward()                        # backward pass to compute and store all of the model's param's gradients
            optimiser.step()                       # update the model's parameters
            
            writer.add_scalar('Loss/Train', loss, epoch*len(train_loader) + idx)    # write loss to a graph

train(epochs)

In [ ]:
def test():
    print('Started evaluation...')
    cnn.eval() #put model into evaluation mode
    
    #calculate the accuracy of our model over the whole test set in batches
    correct = 0
    for x, y in test_loader:
        h = cnn.forward(x)
        pred = h.data.max(1)[1]
        correct += pred.eq(y).sum()
    return correct/len(test_data)

acc = test()
print('Test accuracy: ', acc)